## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/darrenlemyacrawford/Desktop/Class folder/World_Weather_Analysis/Weather_Database/Vacation_Search/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Natchitoches,US,31.7668,-93.1335,88.65,44,86,3.58,overcast clouds
1,1,Karasjok,NO,69.4719,25.5112,57.18,75,100,6.06,overcast clouds
2,2,Komsomolskiy,UZ,40.4272,71.7189,77.72,41,21,4.61,few clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,90,55,8.01,broken clouds
4,4,Bozhou,CN,33.8772,115.7703,73.18,58,97,6.60,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Natchitoches,US,31.7668,-93.1335,88.65,44,86,3.58,overcast clouds
2,2,Komsomolskiy,UZ,40.4272,71.7189,77.72,41,21,4.61,few clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,90,55,8.01,broken clouds
9,9,Saint-Louis,SN,16.3333,-15.0000,85.19,46,10,20.11,clear sky
19,19,Atuona,PF,-9.8000,-139.0333,77.54,65,8,13.38,clear sky
24,24,Fortuna,US,40.5982,-124.1573,76.93,79,100,5.75,overcast clouds
26,26,Bougouni,ML,11.4170,-7.4790,88.88,33,36,7.70,scattered clouds
40,40,Georgetown,MY,5.4112,100.3354,76.95,94,40,3.44,scattered clouds
42,42,Alofi,NU,-19.0595,-169.9187,76.89,94,75,5.75,broken clouds
48,48,Kirakira,SB,-10.4544,161.9205,80.98,80,14,5.01,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                198
City                   198
Country                198
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# .dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Natchitoches,US,88.65,overcast clouds,31.7668,-93.1335,
2,Komsomolskiy,UZ,77.72,few clouds,40.4272,71.7189,
3,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
9,Saint-Louis,SN,85.19,clear sky,16.3333,-15.0000,
19,Atuona,PF,77.54,clear sky,-9.8000,-139.0333,
24,Fortuna,US,76.93,overcast clouds,40.5982,-124.1573,
26,Bougouni,ML,88.88,scattered clouds,11.4170,-7.4790,
40,Georgetown,MY,76.95,scattered clouds,5.4112,100.3354,
42,Alofi,NU,76.89,broken clouds,-19.0595,-169.9187,
48,Kirakira,SB,80.98,few clouds,-10.4544,161.9205,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Natchitoches,US,88.65,overcast clouds,31.7668,-93.1335,Super 8 by Wyndham Natchitoches
2,Komsomolskiy,UZ,77.72,few clouds,40.4272,71.7189,Ikathouse
3,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Saint-Louis,SN,85.19,clear sky,16.3333,-15.0000,
19,Atuona,PF,77.54,clear sky,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
679,Curaca,BR,83.26,overcast clouds,-8.9903,-39.9094,Hotel Recanto Campestre
681,Umm Lajj,SA,88.16,clear sky,25.0213,37.2685,Moon light Furnished Units
683,Jamiltepec,MX,81.72,broken clouds,16.2833,-97.8167,Hotel Cádiz
684,Urucara,BR,75.97,overcast clouds,-2.5364,-57.7600,Pousada Maria Bonita


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]].copy()
locations_df

,Lat,Lng
0,31.7668,-93.1335
2,40.4272,71.7189
3,-0.7393,-90.3518
9,16.3333,-15.0000
19,-9.8000,-139.0333
...,...,...
679,-8.9903,-39.9094
681,25.0213,37.2685
683,16.2833,-97.8167
684,-2.5364,-57.7600


In [17]:
# 11a. Add a marker layer for each city to the map.
locations_df
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations_df, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations_df, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))